## References
> https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard <br>
> https://github.com/trainindata/dmlm-research-and-production/blob/master/Section-2-Machine-Learning-Pipeline-Overview/Machine-Learning-Pipeline-Step2-Feature-Engineering.ipynb

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, StandardScaler

In [2]:
SEED = 0
np.random.seed(SEED)

TARGET = "SalePrice"
TRAIN_SET = "train.csv"
TEST_SET = 'test.csv'

TIME_VARIATIONS = ['year', 'yr']

In [3]:
df_train = pd.read_csv(TRAIN_SET)
df_test = pd.read_csv(TEST_SET)
full_data = pd.concat([df_train, df_test]).reset_index(drop=True)
full_data.set_index('Id', inplace=True)
full_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN


In [4]:
numerical_variables = [var for var in full_data.columns if full_data[var].dtypes != 'O']
# removing target 
numerical_variables.pop(numerical_variables.index(TARGET))
numerical_variables_num = len(numerical_variables)

time_vars = [var for var in numerical_variables for var2 in TIME_VARIATIONS if var2 in var.lower()]

discrete_vars = [var for var in numerical_variables if len(full_data[var].unique()) < 20 and var not in time_vars]
discrete_vars_num = len(discrete_vars)
discrete_vars.pop(discrete_vars.index("PoolArea"));
discrete_vars_num -= 1

continuous_vars = [var for var in numerical_variables if var not in time_vars+discrete_vars]
continuous_vars_num = len(continuous_vars)

categorical_variables = [var for var in full_data.columns if full_data[var].dtypes == 'O']
categorical_variables_num = len(categorical_variables)

In [5]:
print(f"Number of nulls in features: {full_data.isnull().sum().sum()}")

Number of nulls in features: 15424


In [6]:
null_variables_in_numerics = [var for var in numerical_variables if full_data[var].isnull().sum() > 0]
print(f"Number of null features: {len(null_variables_in_numerics)}")
print(f"Number of nulls in numerical features: {full_data[null_variables_in_numerics].isnull().sum().sum()}")
num_nulls = full_data[numerical_variables].isnull().sum().sort_values(ascending=False)
for row in num_nulls.iteritems():
    print(f"{row[0]}  \t{row[1]} nulls \t{row[1]/full_data.shape[0]:.3f}%")

Number of null features: 11
Number of nulls in numerical features: 678
LotFrontage  	486 nulls 	0.166%
GarageYrBlt  	159 nulls 	0.054%
MasVnrArea  	23 nulls 	0.008%
BsmtHalfBath  	2 nulls 	0.001%
BsmtFullBath  	2 nulls 	0.001%
GarageArea  	1 nulls 	0.000%
BsmtFinSF1  	1 nulls 	0.000%
BsmtFinSF2  	1 nulls 	0.000%
BsmtUnfSF  	1 nulls 	0.000%
TotalBsmtSF  	1 nulls 	0.000%
GarageCars  	1 nulls 	0.000%
OverallCond  	0 nulls 	0.000%
YearBuilt  	0 nulls 	0.000%
YearRemodAdd  	0 nulls 	0.000%
OverallQual  	0 nulls 	0.000%
GrLivArea  	0 nulls 	0.000%
LotArea  	0 nulls 	0.000%
1stFlrSF  	0 nulls 	0.000%
2ndFlrSF  	0 nulls 	0.000%
LowQualFinSF  	0 nulls 	0.000%
YrSold  	0 nulls 	0.000%
MoSold  	0 nulls 	0.000%
FullBath  	0 nulls 	0.000%
HalfBath  	0 nulls 	0.000%
BedroomAbvGr  	0 nulls 	0.000%
KitchenAbvGr  	0 nulls 	0.000%
TotRmsAbvGrd  	0 nulls 	0.000%
Fireplaces  	0 nulls 	0.000%
WoodDeckSF  	0 nulls 	0.000%
OpenPorchSF  	0 nulls 	0.000%
EnclosedPorch  	0 nulls 	0.000%
3SsnPorch  	0 nulls 	0.0

In [7]:
null_variables_in_categoric = [var for var in categorical_variables if full_data[var].isnull().sum() > 0]
print(f"Number of null features: {len(null_variables_in_categoric)}")
print(f"Number of nulls in categorical features: {full_data[null_variables_in_categoric].isnull().sum().sum()}")
num_nulls = full_data[categorical_variables].isnull().sum().sort_values(ascending=False)
for row in num_nulls.iteritems():
    print(f"{row[0]}   \t{row[1]} nulls \t{row[1]/full_data.shape[0]:.3f}%")

Number of null features: 23
Number of nulls in categorical features: 13287
PoolQC   	2909 nulls 	0.997%
MiscFeature   	2814 nulls 	0.964%
Alley   	2721 nulls 	0.932%
Fence   	2348 nulls 	0.804%
FireplaceQu   	1420 nulls 	0.486%
GarageCond   	159 nulls 	0.054%
GarageQual   	159 nulls 	0.054%
GarageFinish   	159 nulls 	0.054%
GarageType   	157 nulls 	0.054%
BsmtCond   	82 nulls 	0.028%
BsmtExposure   	82 nulls 	0.028%
BsmtQual   	81 nulls 	0.028%
BsmtFinType2   	80 nulls 	0.027%
BsmtFinType1   	79 nulls 	0.027%
MasVnrType   	24 nulls 	0.008%
MSZoning   	4 nulls 	0.001%
Functional   	2 nulls 	0.001%
Utilities   	2 nulls 	0.001%
Electrical   	1 nulls 	0.000%
KitchenQual   	1 nulls 	0.000%
SaleType   	1 nulls 	0.000%
Exterior2nd   	1 nulls 	0.000%
Exterior1st   	1 nulls 	0.000%
Heating   	0 nulls 	0.000%
Condition1   	0 nulls 	0.000%
Street   	0 nulls 	0.000%
PavedDrive   	0 nulls 	0.000%
LotShape   	0 nulls 	0.000%
LandContour   	0 nulls 	0.000%
LotConfig   	0 nulls 	0.000%
LandSlope   	0 

## Missing values

### Categorical variables

In [8]:
null_variables_in_categoric

['MSZoning',
 'Alley',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType']

In [9]:
# full_data[full_data['MSZoning'].isnull()].shape[0]
# full_data[full_data['Utilities'].isnull()].shape[0]
# full_data[full_data['Exterior1st'].isnull()].shape[0]
# full_data[full_data['Exterior2nd'].isnull()].shape[0]
# full_data[full_data['Electrical'].isnull()].shape[0]
# full_data[full_data['KitchenQual'].isnull()].shape[0]
# full_data[full_data['Functional'].isnull()].shape[0]
# full_data[full_data['SaleType'].isnull()].shape[0]

- **Alley** : data description says NA means "no alley access".
- **MasVnrType** : data description says None means "None".
- **BsmtQual** : data description says NA means "No Basement".
- **BsmtCond** : data description says NA means "No Basement".
- **BsmtExposure** : data description says NA means "No Basement".
- **BsmtFinType1** : data description says NA means "No Basement".
- **BsmtFinType2** : data description says NA means "No Basement".
- **FireplaceQu** : data description says NA means "no fireplace"
- **GarageType** : NA means there is "no garage".
- **GarageFinish** : NA means there is "no garage".
- **GarageQual** : NA means there is "no garage".
- **GarageCond** : NA means there is "no garage".
- **PoolQC** : data description says NA means "No  Pool". 
- **Fence** : data description says NA means "no fence".
- **MiscFeature** : data description says NA means "None".
<br>
<br>
- **MSZoning** : only 4 values missing so fill it with its mode won't affect as much.
- **Utilities** : only 2 values missing so fill it with its mode won't affect as much.
- **Exterior1st & Exterior2nd** : only 1 values missing so fill them with there mode won't affect as much.
- **Electrical** : only 1 values missing so fill it with its mode won't affect as much.
- **KitchenQual** : only 1 values missing so fill it with its mode won't affect as much.
- **Functional** : only 2 values missing so fill it with its mode won't affect as much.
- **SaleType** : only 1 values missing so fill it with its mode won't affect as much.

In [10]:
categorical_features_nulls_filled_with_mode = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'Electrical', 'KitchenQual', 'Functional', 'SaleType']
for var in categorical_features_nulls_filled_with_mode:
    mode = full_data[var].mode()[0]
    full_data[var] = full_data[var].fillna(mode)

full_data[null_variables_in_categoric] = full_data[null_variables_in_categoric].fillna('NONE')

num_nulls = full_data[categorical_variables].isnull().sum().sort_values(ascending=False)
num_nulls

SaleCondition    0
Condition2       0
ExterQual        0
MasVnrType       0
Exterior2nd      0
Exterior1st      0
RoofMatl         0
RoofStyle        0
HouseStyle       0
BldgType         0
Condition1       0
Foundation       0
Neighborhood     0
LandSlope        0
LotConfig        0
Utilities        0
LandContour      0
LotShape         0
Alley            0
Street           0
ExterCond        0
BsmtQual         0
SaleType         0
FireplaceQu      0
MiscFeature      0
Fence            0
PoolQC           0
PavedDrive       0
GarageCond       0
GarageQual       0
GarageFinish     0
GarageType       0
Functional       0
BsmtCond         0
KitchenQual      0
Electrical       0
CentralAir       0
HeatingQC        0
Heating          0
BsmtFinType2     0
BsmtFinType1     0
BsmtExposure     0
MSZoning         0
dtype: int64

### Numerical variables


In [11]:
null_variables_in_numerics

['LotFrontage',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea']

- **LotFrontage** : add a binary missing value indicator variable then fill with the "mode".
- **MasVnrArea** : add a binary missing value indicator variable then fill with the "mode".
- **GarageYrBlt, GarageCars & GarageArea** : since there is no grage then fill with "0".
- **BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath & BsmtHalfBath** : since there is no basement then fill with "0".

In [12]:
numerical_features_nulls_filled_with_zero = ['GarageYrBlt', 'GarageCars', 'GarageArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']
for var in numerical_features_nulls_filled_with_zero:
    full_data[var] = full_data[var].fillna(0)

numerical_features_nulls_filled_with_mode = ['LotFrontage', 'MasVnrArea']
for var in numerical_features_nulls_filled_with_mode:
    mode = full_data[var].mode()[0]
    full_data[var+'_na'] = full_data[var].isnull()
    full_data[var] = full_data[var].fillna(mode)
    
num_nulls = full_data[numerical_variables].isnull().sum().sort_values(ascending=False)
num_nulls   

YrSold           0
MoSold           0
GrLivArea        0
LowQualFinSF     0
2ndFlrSF         0
1stFlrSF         0
TotalBsmtSF      0
BsmtUnfSF        0
BsmtFinSF2       0
BsmtFinSF1       0
MasVnrArea       0
YearRemodAdd     0
YearBuilt        0
OverallCond      0
OverallQual      0
LotArea          0
LotFrontage      0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
WoodDeckSF       0
MiscVal          0
PoolArea         0
ScreenPorch      0
3SsnPorch        0
EnclosedPorch    0
OpenPorchSF      0
GarageArea       0
HalfBath         0
GarageCars       0
GarageYrBlt      0
Fireplaces       0
TotRmsAbvGrd     0
KitchenAbvGr     0
BedroomAbvGr     0
MSSubClass       0
dtype: int64

## Adding Features

- **TimeSold** : the difference between the year the house was sold and the year the house was built.
<!-- - **TimeSold** : the difference between the year the house was sold and the year the house was built. -->

In [13]:
full_data['TimeSold'] = full_data['YrSold'] - full_data['YearBuilt']

## Categorical Encoding

In [14]:
# first we split back the data to its train set and test set
first_record_in_test_set = df_test.iloc[0].Id-1
df_train = full_data.iloc[:first_record_in_test_set]
df_test = full_data.iloc[first_record_in_test_set:]

In [15]:
def to_ordinal_encoding(train, test, var):
    train = train.copy()
    test = test.copy()

    orderd_labels = train.groupby(var)[TARGET].mean().sort_values().index.tolist()
    ordinal_encoder = OrdinalEncoder(categories=[orderd_labels], handle_unknown='ignore')
    
    train[var] = ordinal_encoder.fit_transform(train[var].values.reshape(-1, 1))
    test[var] = ordinal_encoder.transform(test[var].values.reshape(-1, 1))
    
    return train, test

In [16]:
for var in categorical_variables:
    df_train, df_test = to_ordinal_encoding(df_train, df_test, var)

## Feature Transformation & Scaling

In [17]:
def transform_with_log(train, test, features):
    train = train.copy()
    test = test.copy()
    
    for var in features:
        train[var] = np.log(train[var])
        test[var] = np.log(test[var]) 
        
    return train, test

In [18]:
features_transformed_with_log = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']
df_train, df_test = transform_with_log(df_train, df_test, features_transformed_with_log)

In [19]:
features = [var for var in df_train.columns if var != 'SalePrice']

In [20]:
def transform_with_MinMax(train, test, features):
    train = train.copy()
    test = test.copy()
    
    scaler = MinMaxScaler()
    scaler.fit(train[features])
    
    train[features] = scaler.transform(train[features])
    test[features] = scaler.transform(test[features])
    
    return train, test

def transform_with_Standard(train, test, features):
    train = train.copy()
    test = test.copy()
    
    scaler = StandardScaler()
    scaler.fit(train[features])
    
    train[features] = scaler.transform(train[features])
    test[features] = scaler.transform(test[features])
    
    return train, test

In [21]:
df_train, df_test = transform_with_MinMax(df_train, df_test, features)

In [24]:
df_train.to_csv('processed_train.csv', index=False)
df_test.to_csv('processed_test.csv', index=False)